In [7]:
import os

tiny_image = 'TinyImageNet100'
print(os.listdir(tiny_image))

['n02795169', 'n02769748', 'n02917067', 'n01629819', 'n02058221', 'n02793495', 'class_name.txt', 'n02814533', 'n02837789', 'n01770393', 'n01910747', 'n02124075', 'n01774750', 'n02480495', 'n03085013', 'n01443537', 'n03089624', 'n02909870', 'n02977058', 'n02950826', '.DS_Store', 'n02279972', 'n02403003', 'n01742172', 'n01882714', 'n02669723', 'n02226429', 'test', 'n02002724', 'n01768244', 'n02509815', 'n02321529', 'n02099601', 'n02948072', 'n02236044', 'n03126707', 'n02074367', 'n01950731', 'n02268443', 'n02085620', 'n02823428', 'n02113799', 'n01784675', 'n03100240', 'n02788148', 'n02814860', 'n03014705', 'n02481823', 'n01944390', 'n03042490', 'n02190166', 'n02906734', 'n02699494', 'n01855672', 'n02364673', 'n02883205', 'n02815834', 'n02165456', 'n02802426', 'n01641577', 'n01698640', 'n02125311', 'n03179701', 'n02841315', 'n02843684', 'n02437312', 'n02099712', 'n03250847', 'n02094433', 'n02129165', 'n02056570', 'n02410509', 'n03026506', 'n01644900', 'n02666196', 'n02486410', 'n02808440'

In [31]:
import os
import random
from shutil import copy2

def prepare_tinyimagenet_dataset(base_path, num_classes=15):
    """
    Prepare training and testing data from the TinyImageNet100 dataset.

    Parameters:
    - base_path: Path to the TinyImageNet100 dataset.
    - num_classes: Number of classes to select (default is 15).

    Returns:
    - A tuple containing the paths to the training and testing datasets.
    """
    # Ensure the number of classes is at least 15
    assert num_classes >= 15, "Number of classes must be at least 15."

    # Read class labels from class_name.txt
    class_labels_path = os.path.join(base_path, 'class_name.txt')
    if not os.path.isfile(class_labels_path):
        raise FileNotFoundError(f"Class labels file not found: {class_labels_path}")

    with open(class_labels_path, 'r') as f:
        class_labels = [line.strip().split('\t')[0] for line in f.readlines()]

    # Get all class directories
    class_dirs = sorted([d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))])

    # Ensure class directories match those in class_name.txt
    valid_class_dirs = [cls for cls in class_labels if cls in class_dirs]

    # Randomly select the specified number of classes
    selected_classes = random.sample(valid_class_dirs, num_classes)

    # Paths for training and testing datasets
    train_path = os.path.join(base_path, 'train')
    test_path = os.path.join(base_path, 'test')
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)

    for cls in selected_classes:
        cls_image_path = os.path.join(base_path, cls, 'images')
        images = sorted([img for img in os.listdir(cls_image_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))])

        # Prepare training and testing directories for each class
        train_cls_path = os.path.join(train_path, cls)
        test_cls_path = os.path.join(test_path, cls)
        os.makedirs(train_cls_path, exist_ok=True)
        os.makedirs(test_cls_path, exist_ok=True)

        # Copy the first 400 images to training and the next 100 to testing
        for i, img in enumerate(images[:500]):
            src = os.path.join(cls_image_path, img)
            if i < 400:
                dest = os.path.join(train_cls_path, img)
            else:
                dest = os.path.join(test_cls_path, img)
            copy2(src, dest)

    return train_path, test_path

# Example usage
base_path = 'TinyImageNet100'  # Replace with actual path
train_path, test_path = prepare_tinyimagenet_dataset(base_path)
print("Training data prepared at:", train_path)
print("Testing data prepared at:", test_path)


Training data prepared at: TinyImageNet100/train
Testing data prepared at: TinyImageNet100/test


In [33]:
def extract_sift_features(image_path):
    """Extract SIFT features from an image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return keypoints, descriptors

In [39]:
import cv2 
#Extract SIFT features for the first image in the train set
train_image_path = os.path.join(train_path, "class_name", "image.jpg")  # Replace with actual image path
keypoints, descriptors = extract_sift_features(train_image_path)
print(f"Extracted {len(keypoints)} keypoints from the image.")

[ WARN:0@0.009] global loadsave.cpp:241 findDecoder imread_('TinyImageNet100/train/class_name/image.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/features2d/src/sift.dispatch.cpp:512: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'detectAndCompute'
